# gumbel_sinkhorn_ops

> Gumbel-Sinkhorn and Gumbel-matching operators
> Modified from: [https://github.com/perrying/gumbel-sinkhorn](https://github.com/perrying/gumbel-sinkhorn)

Throughout, a permutation $0 \to p_0, 1 \to p_1, ..., n-1 \to p_{n-1}$ is encoded by the permutation matrix $P = (p_{ij})_{i,j=0}^{n-1}$ with $p_{ij} = 1$ if and only if $j = p_i$, and $0$ otherwise.

In NumPy/PyTorch, ``P[arange(n), p]`` is identically equal to ``1``, and we can obtain ``p`` from ``P`` by ``p = P.argmax(-1)``.

In [ ]:
#| default_exp gumbel_sinkhorn_ops

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

from typing import Union

import numpy as np
from scipy.optimize import linear_sum_assignment
import torch


def randperm_mat_like(log_alpha: torch.Tensor) -> torch.Tensor:
    """Generate a random permutation matrix with the same shape as ``log_alpha[-2:]``.
    Assume log_alpha is of shape (*batch_size, n, n)."""
    size = log_alpha.size()[-2:]
    n = size[0]
    rp_mat = torch.zeros(
        size, dtype=log_alpha.dtype, layout=log_alpha.layout, device=log_alpha.device
    )
    rp_mat[torch.arange(n), torch.randperm(n)] = 1

    return rp_mat


def unbias_by_randperms(func: callable) -> callable:
    """Decorator to unbias `func` with two random permutations."""

    def wrapper(log_alpha: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        # Create two random permutation matrices
        rand_perms = (randperm_mat_like(log_alpha), randperm_mat_like(log_alpha))
        # Conjugate log_alpha with the two random permutation matrices
        log_alpha_conj = rand_perms[0] @ log_alpha @ rand_perms[1].mT
        # Apply the function
        out_conj = func(log_alpha_conj, *args, **kwargs)
        # Conjugate back
        out_unconj = rand_perms[0].mT @ out_conj @ rand_perms[1]
        return out_unconj

    return wrapper


def gumbel_noise_like(
    log_alpha: torch.Tensor, noise_factor: float = 1.0, noise_std: bool = False
) -> torch.Tensor:
    """Generate rescaled Gumbel noise with the same shape as log_alpha.
    The noise is rescaled by `noise_factor` or, if `noise_std` is True,
    by `noise_factor` times the standard deviation of log_alpha."""
    uniform_noise = torch.rand_like(log_alpha)
    gumbel_noise = -torch.log(-torch.log(uniform_noise + 1e-20) + 1e-20)
    std = torch.std(log_alpha, dim=(-2, -1), keepdim=True)
    if noise_std:
        noise_factor = noise_factor * std

    return noise_factor * gumbel_noise

In [ ]:
#| export

def sinkhorn_norm(alpha: torch.Tensor, n_iter: int = 20) -> torch.Tensor:
    """Iterative Sinkhorn normalization of non-negative matrices."""
    for _ in range(n_iter):
        alpha = alpha / alpha.sum(-1, keepdim=True)
        alpha = alpha / alpha.sum(-2, keepdim=True)

    return alpha


@torch.compile
def log_sinkhorn_norm(log_alpha: torch.Tensor, n_iter: int = 20) -> torch.Tensor:
    """Iterative Sinkhorn normalization in log space, for numerical stability."""
    for _ in range(n_iter):
        log_alpha = log_alpha - torch.logsumexp(log_alpha, -1, keepdim=True)
        log_alpha = log_alpha - torch.logsumexp(log_alpha, -2, keepdim=True)

    return log_alpha


def gumbel_sinkhorn(
    log_alpha: torch.Tensor,
    *,
    tau: Union[float, torch.Tensor] = 1.0,
    n_iter: int = 10,
    noise: bool = False,
    noise_factor: float = 1.0,
    noise_std: bool = False,
) -> torch.Tensor:
    """Gumbel-Sinkhorn operator with a temperature parameter `tau`.
    Given arbitrary square matrices, outputs bistochastic matrices that are close to
    permutation matrices when `tau` is small."""
    if noise:
        log_alpha = log_alpha + gumbel_noise_like(
            log_alpha, noise_factor=noise_factor, noise_std=noise_std
        )
    log_alpha = log_alpha / tau
    bistochastic_mats = torch.exp(log_sinkhorn_norm(log_alpha, n_iter))

    return bistochastic_mats

In [ ]:
show_doc(gumbel_sinkhorn)

In [ ]:
#| export

def np_matching(cost: np.ndarray) -> np.ndarray:
    """Find an assignment matrix with maximum cost, using the Hungarian algorithm.
    Return the matrix in dense format."""
    row, col = linear_sum_assignment(cost, maximize=True)
    np_matching_mat = np.zeros_like(cost)
    np_matching_mat[row, col] = 1

    return np_matching_mat


def matching(
    log_alpha: torch.Tensor,
) -> torch.Tensor:
    np_log_alpha = log_alpha.detach().clone().cpu().numpy()
    np_matching_mats = np.zeros_like(np_log_alpha)
    for idx in np.ndindex(np_log_alpha.shape[:-2]):
        np_matching_mats[idx] = np_matching(np_log_alpha[idx])
    matching_mats = (
        torch.from_numpy(np_matching_mats).to(log_alpha.device).to(log_alpha.dtype)
    )

    return matching_mats


def gumbel_matching(
    log_alpha: torch.Tensor,
    *,
    noise: bool = False,
    noise_factor: float = 1.0,
    noise_std: bool = False,
    unbias_lsa: bool = False,
) -> torch.Tensor:
    """Gumbel-matching operator, i.e. the solution of the linear assignment problem with
    optional Gumbel noise."""
    if noise:
        log_alpha = log_alpha + gumbel_noise_like(
            log_alpha, noise_factor=noise_factor, noise_std=noise_std
        )
    gumbel_matching_impl = unbias_by_randperms(matching) if unbias_lsa else matching
    assignment_mat = gumbel_matching_impl(log_alpha)

    return assignment_mat

In [ ]:
show_doc(gumbel_matching)

In [ ]:
#| export

def inverse_permutation(x: torch.Tensor, mats: torch.Tensor) -> torch.Tensor:
    """When mats contains permutation matrices, exchange the rows of `x` using the inverse(s)
    of the permutation(s) encoded in `mats`."""
    return mats.mT @ x.to(mats.dtype)